In [167]:
%run -i header.py
from sympy import N
from sympy.physics.quantum.cg import CG
from scipy.special import sph_harm, spherical_jn

In [170]:
def clebsch(a, b, c, d, e ,f):
    return sp.array(N(CG(a, b, c, d, e, f).doit()), dtype=sp.complex64)

Define polarization vectors:

- $\lambda = \pm1:\qquad \vec{e}_\lambda = -\frac{\lambda}{\sqrt{2}} \{ 1, \mathrm{i} \lambda, 0 \}$
- $\lambda = 0:\qquad \vec{e}_\lambda = \{ 0, 0, 1 \}$

In [160]:
def ee(lam):
    return -lam/sp.sqrt(2)*sp.array([1, lam*1j, 0], dtype=sp.complex64) + sp.array([0, 0, 1-lam**2])

Define vector spherical harmonic:

$\vec{Y}_{l, J}^\lambda = \sum_{m_1, m_2} <J \lambda|l m_1 1 m_2> Y_L^{m_1} \vec{e}_{m_2}$

In [171]:
def Yvec(l, J, m, th, phi):
    res = 0
    for m1 in range(-l, l+1, 1):
        for m2 in range(-1, 2, 1): 
            res += sph_harm(m1, l, phi, th)*clebsch(l, m1, 1, m2, J, m)*ee(m2)
    return  res

Spherical harmonic of the order 0 doesn't depend on $\phi$ in general notation. It is represented in notation of SciPy `sph_harm`.

Adaptation scheme: $Y_l^m(\theta, \phi)$ = `sph_harm`$(m, l, \phi, \theta)$

In [172]:
sph_harm(0, 5, 3.2, 0.2) - sph_harm(0, 5, 2.4, 0.2)

0j

Plane wave expansion according to [Wiki](https://en.wikipedia.org/wiki/Plane_wave_expansion).

We apply the following property of spherical harmonics:

$Y_l^0(\theta, \phi) = \sqrt{\frac{2l+1}{4 \pi}} P_l(\cos(\theta))$

In [173]:
def testLegendreSph(n, th):
    return sph_harm(0, n, 0, th) - sp.sqrt((2*n+1)/4/sp.pi)*sp.special.legendre(n)(sp.cos(th))
testLegendreSph(4,0.2)

(-3.3306690738754696e-16+0j)

In [174]:
def pwExp(l, kr, th):
    return sp.sqrt(4*sp.pi*(2*l+1))*1j**l*spherical_jn(l, kr)*sph_harm(0, l, 0, th)
def pwExpSum(lmax, kr, th):
    return pwExp(sp.arange(0, lmax, 1), kr, th).sum()

In [219]:
pwExpSum(10, 1.3, 0.2) - sp.exp(1j*1.3*sp.cos(0.2))

(3.3868731708075472e-09+7.025391379755774e-11j)

In [206]:
def testClebschOrtho(j1, j2, M1, M2):
    return sum([ clebsch(j1, M1[0], j2, M1[1], j, sum(M1))*clebsch(j1, M2[0], j2, M2[1], j, sum(M2)) for j in range(abs(j1-j2), (j1+j2) +1, 1)])
testClebschOrtho(1, 2, (1,2), (1,3))

0j

In [210]:
def testOrtho(J, lam, th):
    return sph_harm(0, J, 0, th)*ee(lam) - sum([Yvec(J, j, lam, th, 0)*clebsch(J, 0, 1, lam, j, lam) for j in range(abs(J-1), J+2, 1)])
testOrtho(2, 1, 0.3)

array([-9.64022001e-09+0.00000000e+00j,  0.00000000e+00-8.83757939e-09j,
       -4.75679567e-09+0.00000000e+00j])

In [233]:
def prepremult(jmax, lam, kr, th):
   return sum([\
        1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J,lam)*Yvec(J,J,lam,th,0)\
       for J in range(1, jmax+1, 1)])\
   +sum([\
        1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J+1,lam)*Yvec(J,J+1,lam,th,0)\
       for J in range(0, jmax+1, 1)])\
   +sum([\
        1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J-1,lam)*Yvec(J,J-1,lam,th,0)\
       for J in range(1, jmax+1, 1)])
display(prepremult(10, 1, 1, 0.2) - ee(1)*sp.exp(1j*sp.cos(0.2)))

array([-4.92803814e-09+1.97833974e-08j, -1.98446799e-08-4.73346518e-09j,
       -1.57252362e-09-5.00387396e-11j])

In [242]:
def premult(jmax, lam, kr, th):
   return sum([\
        1j**J*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J, kr)*clebsch(J,0,1,lam,J,lam)*Yvec(J,J,lam,th,0)\
       for J in range(1, jmax+1, 1)])\
   +sum([\
        -1j**(J+1)*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J-1, kr)*clebsch(J-1,0,1,lam,J,lam)*Yvec(J-1,J,lam,th,0)*sp.sqrt((2*J-1)/(2*J+1))\
       for J in range(1, jmax+1, 1)])\
   +sum([\
        1j**(J+1)*sp.sqrt(4*sp.pi*(2*J+1))*spherical_jn(J+1, kr)*clebsch(J+1,0,1,lam,J,lam)*Yvec(J+1,J,lam,th,0)*sp.sqrt((2*J+3)/(2*J+1))\
       for J in range(0, jmax+1, 1)])
display(premult(15, 1, 1, 0.2) - ee(1)*sp.exp(1j*sp.cos(0.2)))

array([-4.92793162e-09+1.97861991e-08j, -1.98474815e-08-4.73335865e-09j,
       -1.57252361e-09-5.00387553e-11j])

In [248]:
def mult(J, lam, kr, th):
    return -sp.sqrt(2*sp.pi)*(1j)**J*sp.sqrt(2*J+1)*(lam*spherical_jn(J, kr)*Yvec(J, J, lam, th, 0.)+1j*(sp.sqrt((J+1)/(2*J+1))*spherical_jn(J-1, kr)*Yvec(J-1, J, lam, th, 0) - sp.sqrt(J/(2*J+1))*spherical_jn(J+1, kr)*Yvec(J+1, J, lam, th, 0)))

In [251]:
sum([mult(J, 1, 1, 0.2) for J in range(1,15)]) - ee(1)*sp.exp(1j*1*sp.cos(0.2))

array([-2.49798704e-09+9.87284032e-09j, -9.94399985e-09-2.33265757e-09j,
       -1.84763736e-09+4.23745651e-11j])

In [252]:
def mult(J, lam, kr, th):
    return -sp.sqrt(2*sp.pi)*(-1j)**J*sp.sqrt(2*J+1)*(lam*spherical_jn(J, kr)*Yvec(J, J, lam, th, 0.)-1j*(sp.sqrt((J+1)/(2*J+1))*spherical_jn(J-1, kr)*Yvec(J-1, J, lam, th, 0) - sp.sqrt(J/(2*J+1))*spherical_jn(J+1, kr)*Yvec(J+1, J, lam, th, 0)))
sum([mult(J, 1, 1, 0.2) for J in range(1,15)]) - ee(1)*sp.exp(-1j*1*sp.cos(0.2))

array([-2.49798704e-09-9.87284032e-09j,  9.94399985e-09-2.33265757e-09j,
       -1.84763736e-09-4.23745651e-11j])

Result: Multipole expansion works as in the thesis

## Coeff Q symmetry

In [279]:
def coefQ(J, lam, instate, outstate):
    return sum([clebsch(outstate["L"], outstate["m"]-s, outstate["S"], s, outstate["J"], outstate["m"])*\
         clebsch(instate["L"], instate["m"]-s, instate["S"], s, instate["J"], instate["m"])*\
         clebsch(J, -lam, instate["L"], instate["m"]-s, outstate["L"], outstate["m"]-s)
        for s in range(-instate["S"], instate["S"]+1, 1)])**2
    
a1 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})# chi_c2 -> psi_1S
a2 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1}) #chi_c2 -> psi_1S
display(a1/a2)

b1 = coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":1, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":-1, "L":1, "S":1})# psi_3S -> chi_c2
b2 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":2, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":-1, "L":0, "S":1}, {"J":2, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":-1, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":2, "m":1, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":2, "m":-2, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":1, "L":0, "S":1}, {"J":2, "m":2, "L":1, "S":1})
display(b1/b2)

a1 = coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})# chi_c2 -> psi_1S
a2 = coefQ(1, -1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":0, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-2, "L":1, "S":1}, {"J":1, "m":-1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":2, "L":1, "S":1}, {"J":1, "m":1, "L":0, "S":1})+\
    coefQ(1, 1, {"J":2, "m":1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1})+\
    coefQ(1, -1, {"J":2, "m":-1, "L":1, "S":1}, {"J":1, "m":0, "L":0, "S":1}) #chi_c2 -> psi_1S
display(a1/a2)


(0.10000000506295846+0j)

(0.29999999062850013+0j)

(0.6000000040888882+0j)

In [280]:
b1 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})# psi_3S -> chi_c2
b2 = coefQ(1, 1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":0, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":0, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, 1, {"J":1, "m":-1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})+\
    coefQ(1, -1, {"J":1, "m":1, "L":0, "S":1}, {"J":0, "m":0, "L":1, "S":1})
display(b1/b2)

(0.29999999062850013+0j)